In [ ]:
from keras.layers import Input, Dense, Activation, Cropping2D, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Concatenate, Add
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.models import Model
from keras.utils import layer_utils, plot_model, to_categorical, multi_gpu_model, normalize
import keras.backend as K
from PIL import Image
import numpy as np
import keras
#import models
from matplotlib import pyplot as plt
from keras.optimizers import Adam, rmsprop
import math
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os

#Import and rescale dataset

importdataset=np.memmap('MODIS_Sc_scenes/numpy_arrays/resized_numpy_images.npy')
dataset = 2*(importdataset/255.)-1

print('dataset dtype = ', dataset.dtype)
print('dataset shape = ', dataset.shape)

print('Done!')

In [ ]:
#defining the model for creating the rough mask

import cv2
import numpy as np
import copy

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Flatten, Activation, add
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.engine import Layer, InputSpec
from keras import backend as K

import sys
sys.setrecursionlimit(3000)

trainable = 'False'

class Scale(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializers](../initializers.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = K.variable(self.gamma_init(shape), name='%s_gamma'%self.name)
        self.beta = K.variable(self.beta_init(shape), name='%s_beta'%self.name)
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), name=conv_name_base + '2a', use_bias=False, trainable=trainable)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size), name=conv_name_base + '2b', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = add([x, input_tensor], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), strides=strides, name=conv_name_base + '2a', use_bias=False, trainable=trainable)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size),
                      name=conv_name_base + '2b', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Conv2D(nb_filter3, (1, 1), strides=strides,
                             name=conv_name_base + '1', use_bias=False, trainable=trainable)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = add([x, shortcut], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def resnet152_model(n_classes, weights_path=None):
    '''Instantiate the ResNet152 architecture,
    # Arguments
        weights_path: path to pretrained weight file
    # Returns
        A Keras model instance.
    '''
    eps = 1.1e-5

    # Handle Dimension Ordering for different backends
    global bn_axis
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
        img_input = Input(shape=(224, 224, 3), name='data')
    else:
        bn_axis = 1
        img_input = Input(shape=(3, 224, 224), name='data')
            
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', use_bias=False, trainable=trainable)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1,8):
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1,36):
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    #for classification
    #x_fc = AveragePooling2D((7, 7), name='avg_pool')(x)
    #x_fc = Flatten()(x_fc)
    #x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)

    #new section for segmentation
    up1 = UpSampling2D((2,2))(x) #7 -> 14
    #up1 = ZeroPadding2D(((0,2),(0,2)))(up1) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden1 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up1)#concat1
    conv1 = BatchNormalization()(up1)#concat1
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    add1 = Add()([iden1,conv1])
    
    up2 = UpSampling2D((4,4))(add1) #14 -> 56
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden2 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up2)#concat2
    conv2 = BatchNormalization()(up2)#concat2
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    add2 = Add()([iden2,conv2])
    
    up3 = UpSampling2D((4,4))(add2) #56 -> 224
    #up2 = ZeroPadding2D(((0,2),(0,2)))(up2) #160->162
    #concat1 = Concatenate(axis=3)([up7,add3])
    iden3 = Conv2D(smallest_layer*4, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(up3)#concat3
    conv3 = BatchNormalization()(up3)#concat3
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(smallest_layer*4, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv3) #200
    add3 = Add()([iden3,conv3])
    
    conv4 = Conv2D(n_classes, 3, activation ='sigmoid', padding = 'same', kernel_initializer = 'he_normal')(add3)
    
    model = Model(img_input, conv4)
    
     # load weights
    if weights_path:
        model.load_weights(weights_path, by_name=True)

    return model

In [ ]:
#define the model for refining the mask

K.set_image_data_format('channels_last')
K.set_floatx('float32')
smallest_layer=32

def RESUNETMEDIUM(input_shape, n_classes):

    inputs = Input(input_shape)

    ###encoding block 1
    iden1 = Conv2D(smallest_layer, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(inputs) #200
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv1) #200
    add1 = Add()([iden1,conv1])
    pool1 = MaxPooling2D((3,3))(add1) #224 -> 74
    print(add1)

    ###encoding block2
    iden2 = Conv2D(smallest_layer*2, 1, activation = None, padding='same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(pool1)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv2) #200
    add2 = Add()([iden2,conv2])
    pool2 = MaxPooling2D((3,3))(add2) #74 -> 24
    print (add2)

    ###encoding block4
    iden4 = Conv2D(smallest_layer*8, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(pool2)
    conv4 = BatchNormalization()(pool2)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv4) #200
    add4 = Add()([iden4,conv4])
    drop4 = Dropout(0.5)(add4)
    pool4 = MaxPooling2D((3,3))(drop4) #24 -> 8
    print (pool4)

    ###bridge
    conv5 = BatchNormalization()(pool4)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(smallest_layer*16, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv5) #200
    drop5 = Dropout(0.5)(conv5)
    print (conv5)

    ###decoding block1
    up6 = UpSampling2D((3,3))(drop5) #8->24
    #up6 = ZeroPadding2D(((1,0),(1,0)))(up6) #24->25
    concat6 = Concatenate(axis=3)([up6,add4])
    iden6 = Conv2D(smallest_layer*8, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat6)
    conv6 = BatchNormalization()(concat6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv6) #200
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(smallest_layer*8, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv6) #200
    add6 = Add()([iden6,conv6])

    ###decoding block3
    up8 = UpSampling2D((3,3))(add6) #24 -> 72
    up8 = ZeroPadding2D(((0,2),(0,2)))(up8) #72->74
    concat8 = Concatenate(axis=3)([up8,add2])
    iden8 = Conv2D(smallest_layer*2, 1, activation=None, padding='same', kernel_initializer = 'he_normal')(concat8)
    conv8 = BatchNormalization()(concat8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv8) #200
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(smallest_layer*2, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv8) #200
    add8 = Add()([iden8,conv8])

    ###decoding block4
    up9 = UpSampling2D((3,3))(add8) #74 -> 222
    up9 = ZeroPadding2D(((0,2),(0,2)))(up9) #222->224
    concat9 = Concatenate(axis=3)([up9,add1])
    iden9 = Conv2D(smallest_layer,1,activation=None, padding='same', kernel_initializer = 'he_normal')(concat9)
    conv9 = BatchNormalization()(concat9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(smallest_layer, 3, activation = None, padding = 'same', kernel_initializer = 'he_normal')(conv9) #200
    add9 = Add()([iden9,conv9])

    conv10 = Conv2D(n_classes, 3, activation ='sigmoid', padding = 'same', kernel_initializer = 'he_normal')(add9)
    #sigmoid probably too strong an activation

    model = Model(input = inputs, output = conv10)

    return model


In [ ]:
#define the dice coefficient used
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [ ]:
#create the rough mask model and load the weights
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3,4,5,6,7,8"
no_gpus=1
smallest_layer=32
if no_gpus > 1:
    rough_mask_model=multi_gpu_model(resnet152_model(1), gpus=no_gpus)

else:
    rough_mask_model=resnet152_model(1)


from keras.metrics import categorical_accuracy
optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
loss=dice_coef_loss
rough_mask_model.compile(optimizer=optimizer,
                         loss=loss, metrics=[dice_coef, 'accuracy', 'binary_crossentropy'])

rough_mask_model.load_weights('model_weights/rough_model_weights.h5')

In [ ]:
#create the refined mask model and load the weights
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2,3,4,5,6,7,8"
no_gpus=1
smallest_layer=32
if no_gpus > 1:
    refined_mask_model=multi_gpu_model(RESUNETMEDIUM((224,224,3), 1), gpus=no_gpus)
else:
    refined_mask_model=RESUNETMEDIUM((224,224,3), 1)

from keras.metrics import categorical_accuracy
optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
loss=dice_coef_loss
refined_mask_model.compile(optimizer=optimizer,
                         loss=loss, metrics=[dice_coef, 'accuracy', 'binary_crossentropy'])

refined_mask_model.load_weights('refine_model_weights.h5')

In [ ]:
from keras.utils import plot_model
print(refined_mask_model.summary())
print(rough_mask_model.summary())
plot_model(refined_mask_model, to_file='refined_mask_model.pdf')
plot_model(rough_mask_model, to_file='rough_mask_model.pdf')

In [ ]:
#create the masks and dilate them, then apply them to the dataset

mask = rough_mask_model.predict(dataset, batch_size=4*no_gpus)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6))

dilated_mask = np.zeros(mask.shape)
for i in range(mask.shape[0]):
    dilated_mask[i,...,0]=cv2.dilate(mask[i,...], kernel, iterations=6)

dataset_masked = np.zeros(dataset.shape)
for i in range(3):
    dataset_masked[...,i]=(dataset[...,i]+1)/2*dilated_mask[...,0]

In [ ]:
#refine the masks

refined_mask = refined_mask_model.predict(dataset_masked, batch_size=no_gpus*4)

In [ ]:
#print some statistics about the model
tru = resized_labels.reshape(resized_labels.shape[0],224,224,1).astype('bool')
pre = refined_mask>=0.5

true_positives = tru*pre
false_positives = pre*~tru
true_negatives = ~tru*~pre
false_negatives = tru*~pre

tot_tp = np.sum(true_positives)
tot_fp = np.sum(false_positives)
tot_tn = np.sum(true_negatives)
tot_fn = np.sum(false_negatives)

precision = tot_tp/(tot_tp+tot_fp)
recall = tot_tp/(tot_tp+tot_fn)
f1 = 2*precision*recall/(precision + recall)
TPR = recall #recall is the same as true positive rate
TNR = tot_tn/(tot_tn+tot_fp) #true negative rate
bACC = (TPR+TNR)/2 #Balanced accuracy score

print('pixel precision = ', precision)
print('pixel recall = ', recall)
print('pixel f1 score = ', f1)
print('pixel true positive rate = ', TPR)
print('pixel true negative rate = ', TNR)
print('pixel balanced accuracy score = ', bACC)

In [ ]:
pre = pre[...,0].any(axis=(1,2))
tru = tru[..., 0].any(axis=(1,2))

true_positives = tru*pre
false_positives = pre*~tru
true_negatives = ~tru*~pre
false_negatives = tru*~pre

tot_tp = np.sum(true_positives)
tot_fp = np.sum(false_positives)
tot_tn = np.sum(true_negatives)
tot_fn = np.sum(false_negatives)

precision = tot_tp/(tot_tp+tot_fp)
recall = tot_tp/(tot_tp+tot_fn)
f1 = 2*precision*recall/(precision + recall)
TPR = recall #recall is the same as true positive rate
TNR = tot_tn/(tot_tn+tot_fp) #true negative rate
bACC = (TPR+TNR)/2 #Balanced accuracy score

print('image precision = ', precision)
print('image recall = ', recall)
print('image f1 score = ', f1)
print('image true positive rate = ', TPR)
print('image true negative rate = ', TNR)
print('image balanced accuracy score = ', bACC)
    

In [ ]:
# Display all the images that either have pocs, or have had pocs detected

poc_presence = np.logical_or(tru[..., 0].any(axis=(1,2)), pre[..., 0].any(axis=(1,2)))
original_images = resized_dataset[poc_presence]
samples = datasetmasked[poc_presence]
annotations = resized_labels[poc_presence]
prediction = pre[poc_presence]


f = plt.figure(figsize=(18,900))
for image, _ in enumerate(samples):
    f.add_subplot(samples.shape[0],4, 4*image+1)
    plt.imshow(np.clip(original_images[image,:,:,:]/2+0.5, 0, 1))
    f.add_subplot(samples.shape[0],4, 4*image+2)
    plt.imshow(np.clip(samples[image,:,:,:], 0, 1))
    f.add_subplot(samples.shape[0],4, 4*image+3)
    plt.imshow(annotations[image,:,:])
    f.add_subplot(samples.shape[0],4, 4*image+4)
    plt.imshow(prediction[image,:,:,0])
f.tight_layout()
f.subplots_adjust(hspace=0.1)
plt.show(block=True)